# PDF Loader

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from app.components.utils import load_env
env = load_env()

In [ ]:
from app.components.media import PDFLoader
pdf_loader = PDFLoader()
# documents = pdf_loader.load_and_summarize(
    # "/workspaces/FoodRAG/assets/healthy_nutrition/10 healthier food options at hawker centres _ The Straits Times.pdf")

In [ ]:
docs_text = pdf_loader.load(
    # "/workspaces/FoodRAG/assets/healthy_nutrition/101 Guide to Healthy Eating (English).pdf")
    "/workspaces/FoodRAG/assets/healthy_nutrition/10 healthier food options at hawker centres _ The Straits Times.pdf")
    # "/workspaces/FoodRAG/assets/healthy_nutrition/all-about-your-diet-booklet-for-prostate-cancer-patients.pdf")
    # "/workspaces/FoodRAG/assets/healthy_nutrition/Early chilhood nutrition from 1-2 years old.pdf")

In [ ]:
len(docs_text)

In [ ]:
print(docs_text[1].page_content)

In [ ]:
print("\n".join([doc.page_content for doc in docs_text]))

In [ ]:
import pytesseract
import re


def detect_image_lang(img_path):
    osd = pytesseract.image_to_osd(img_path,output_type=pytesseract.Output.DICT)
    return osd['script']


detect_image_lang("/tmp/image_3.png")

In [ ]:
docs_reviser = pdf_loader.remove_unwanted_text(docs_text)

In [ ]:
print("\n".join([doc.page_content for doc in docs_reviser]))

In [ ]:
from app.components.media import ImageLoader
image_loader = ImageLoader()

In [ ]:
print(image_loader.load("/tmp/image_1.png"))

In [ ]:
import IPython.display as display
display.Image("/tmp/image_1.png")

# PDF Database

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from app.components.database import BaseDatabaseToolkit
pdf_db = BaseDatabaseToolkit(sql_database_url="sqlite:///database/usda.db")

/home/vscode/.local/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [ ]:
from langchain_core.messages import HumanMessage
pdf_db.invoke({"messages":[
  HumanMessage("What is the nutrition of banana"),]})

In [ ]:
pdf_db.get_tools()

In [ ]:
from app.components.database import BaseVectorDatabaseToolkit
from chromadb import HttpClient
vector_db = HttpClient(host="chroma")

In [ ]:
food_vector = BaseVectorDatabaseToolkit(vector_db)
food_vector.get_tools()

In [ ]:
from langchain_core.messages import HumanMessage
answ = food_vector.invoke({"messages":[
  HumanMessage("Can you save audio file?"),]})
for a in answ['messages']:
    print(a)

# Agent

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from langchain.globals import set_debug
set_debug(True)

In [ ]:
from app.components.agent import FoodDataAgent

In [ ]:
agent = FoodDataAgent(pdf_db, food_vector)

In [ ]:
agent.invoke({"messages":[
  HumanMessage("What is the nutrition of banana from sql?"),]})